In [18]:
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
data_foot = pd.read_csv("data-es (1).csv", sep=";")

In [20]:
data_rugby = pd.read_csv("data-es-rugby.csv", sep=";")

In [21]:
data_mixte = pd.read_csv("data-es_mixte.csv", sep=";")

In [22]:
#jointure data_foot et data_rugby

data = pd.concat([data_foot, data_rugby])

In [23]:
data = pd.concat([data, data_mixte])

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41475 entries, 0 to 2251
Columns: 113 entries, Numéro de l'équipement sportif to gen_2024Fin_Labellisation
dtypes: bool(17), float64(25), int64(2), object(69)
memory usage: 31.4+ MB


In [30]:
data_pop = pd.read_excel('POPULATION_MUNICIPALE_COMMUNES_FRANCE.xlsx')
data_pop.head()


,objectid,reg,dep,cv,codgeo,libgeo,p13_pop,p14_pop,p15_pop,p16_pop,p17_pop,p18_pop,p19_pop,p20_pop,p21_pop
0,115658,52,85,8502,85062,Châteauneuf,968.0,993.0,1013.0,1027.0,1056,1085.0,1114.0,1118.0,1134.0
1,115659,26,58,5808,58300,Urzy,1839.0,1835.0,1828.0,1802.0,1775,1749.0,1746.0,1747.0,1742.0
2,115660,43,70,7012,70137,Chassey-lès-Montbozon,218.0,217.0,216.0,215.0,217,215.0,215.0,220.0,225.0
3,115661,21,51,5123,51649,Vitry-le-François,13174.0,13144.0,12805.0,12552.0,12133,11743.0,11376.0,11458.0,11454.0
4,115662,11,78,7811,78638,Vaux-sur-Seine,4749.0,4715.0,4788.0,4857.0,4927,4929.0,5010.0,5020.0,5083.0


In [31]:
# Harmoniser les formats
data["Commune INSEE"] = data["Commune INSEE"].astype(str).str.zfill(5)
data_pop["codgeo"] = data_pop["codgeo"].astype(str).str.zfill(5)

# Jointure commune ↔ population
df_joined = data.merge(data_pop[["codgeo", "p21_pop"]], 
                       left_on="Commune INSEE", 
                       right_on="codgeo",
                       how="left")


In [34]:
df_joined.head()

,Numéro de l'équipement sportif,Numéro de l'installation sportive,Date de l'enquête,Nom de l'installation sportive,SIRET Installation,Adresse,Code Postal,Commune nom,Commune INSEE,Type de particularité de l'installation,...,Longitude,Latitude,Type de particularité de l'installation (brute),Activités,QPV,QPV à 200 mètres,Catégorie,gen_2024Fin_Labellisation,codgeo,p21_pop
0,E002I774820003,I774820003,2025-03-31,STADE MUNICIPAL,NaN,RUE DU STADE,77130,Varennes-sur-Seine,77482,NaN,...,2.925389,48.377422,NaN,Football / Football en salle (Futsal),NaN,NaN,structurant,NaN,77482,3688.0
1,E002I774870005,I774870005,2025-03-31,STADE MUNICIPAL,NaN,LA MARE AUX CHAMPS,77000,Vaux-le-Pénil,77487,NaN,...,2.681070,48.529500,NaN,Football / Football en salle (Futsal),NaN,NaN,structurant,NaN,77487,11174.0
2,E002I774930001,I774930001,2025-03-31,COMPLEXE SPORTIF HENRI BARBERAN,NaN,RUE ALBERT HUBSCHWERLIN,77390,Verneuil-l'Étang,77493,Complexe sportif,...,2.833150,48.648610,"[""Complexe sportif""]",Football / Football en salle (Futsal),NaN,NaN,structurant,NaN,77493,3198.0
3,E002I775120005,I775120005,2025-03-31,STADE DU FOURCHERET,NaN,LE FOURCHERET,77510,Villeneuve-sur-Bellot,77512,NaN,...,3.325275,48.860313,NaN,Football / Football en salle (Futsal),NaN,NaN,structurant,NaN,77512,1136.0
4,E002I775140008,I775140008,2025-03-31,PARC DES SPORTS DES PETITS MARAIS,NaN,CHEMIN DES PETITS MARAIS,77270,Villeparisis,77514,NaN,...,2.624140,48.944070,NaN,Football / Football en salle (Futsal),NaN,NaN,structurant,NaN,77514,26822.0


In [36]:
df_joined["Type d'équipement sportif"].unique()

array(['Terrain de football', 'Terrain de rugby', 'Terrain mixte'],
      dtype=object)

In [37]:
df_joined = df_joined[df_joined["Région Nom"] == "Occitanie"]
df_joined = df_joined[df_joined["p21_pop"] < 5000]
mask_rugby = (
    df_joined["Type d'équipement sportif"].str.contains("RUGBY", case=False, na=False)
    | df_joined["Activités"].str.contains("Rugby", case=False, na=False)
)

df_rugby = df_joined[mask_rugby].copy()

In [38]:
communes_small = (
    df_joined[["codgeo", "Commune INSEE", "Commune nom", "Longitude", "Latitude", "p21_pop"]]
    .drop_duplicates(subset=["codgeo"])
)
rugby_fields = df_rugby[["Longitude", "Latitude"]].dropna().drop_duplicates()


In [39]:
import numpy as np

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371  # rayon de la Terre (km)
    return R * c


In [40]:
dist_min_list = []

for idx, row in communes_small.iterrows():
    dists = haversine(
        row["Longitude"],
        row["Latitude"],
        rugby_fields["Longitude"].values,
        rugby_fields["Latitude"].values
    )
    dist_min_list.append(dists.min())
    
communes_small["distance_min_km"] = dist_min_list


In [47]:
communes_small.head()

,codgeo,Commune INSEE,Commune nom,Longitude,Latitude,p21_pop,distance_min_km
107,81001,81001,Aguts,1.920320,43.527630,236.0,8.086603
110,81038,81038,Brens,1.919380,43.885110,2393.0,0.088444
111,81046,81046,Cadalen,1.991880,43.851280,1531.0,0.081480
112,81052,81052,Cambon,2.213340,43.917880,2129.0,0.099486
113,81054,81054,Cambounet-sur-le-Sor,2.111514,43.577998,971.0,3.868072


In [49]:
communes_clean = communes_small.dropna(subset=["distance_min_km", "p21_pop"])

mean_distance = communes_clean["distance_min_km"].mean()

weighted_mean_distance = np.average(
    communes_clean["distance_min_km"],
    weights=communes_clean["p21_pop"]
)

print(f"Distance moyenne (non pondérée) : {mean_distance:.2f} km")
print(f"Distance moyenne pondérée par la population : {weighted_mean_distance:.2f} km")



Distance moyenne (non pondérée) : 6.48 km
Distance moyenne pondérée par la population : 5.37 km


In [52]:
import numpy as np

# On enlève les NaN distance / population
communes_clean = communes_small.dropna(subset=["distance_min_km", "p21_pop"])

dist = communes_clean["distance_min_km"].values
pop = communes_clean["p21_pop"].values

w_mean = np.average(dist, weights=pop)
w_var = np.average((dist - w_mean) ** 2, weights=pop)
w_std = np.sqrt(w_var)

def weighted_quantile(values, weights, quantiles):
    sorter = np.argsort(values)
    values = values[sorter]
    weights = weights[sorter]
    cumsum = np.cumsum(weights)
    total = cumsum[-1]
    qs = []
    for q in quantiles:
        qs.append(values[np.searchsorted(cumsum, q * total)])
    return qs

q25, q50, q75 = weighted_quantile(dist, pop, [0.25, 0.5, 0.75])

w_min = np.min(dist)
w_max = np.max(dist)

print("=== Statistiques pondérées par population ===")
print(f"Count : {len(dist)}")
print(f"Mean : {w_mean:.3f} km")
print(f"Std : {w_std:.3f} km")
print(f"Min : {w_min:.3f} km")
print(f"25% : {q25:.3f} km")
print(f"Median (50%) : {q50:.3f} km")
print(f"75% : {q75:.3f} km")
print(f"Max : {w_max:.3f} km")


=== Statistiques pondérées par population ===
Count : 1531
Mean : 5.373 km
Std : 5.955 km
Min : 0.000 km
25% : 0.097 km
Median (50%) : 3.734 km
75% : 8.678 km
Max : 30.076 km


In [9]:
data.columns.tolist()

["Numéro de l'équipement sportif",
 "Numéro de l'installation sportive",
 "Date de l'enquête",
 "Nom de l'installation sportive",
 'SIRET Installation',
 'Adresse',
 'Code Postal',
 'Commune nom',
 'Commune INSEE',
 "Type de particularité de l'installation",
 'Unité Administrative Immatriculée (UAI)',
 "Accessibilité de l'installation en fonction du type handicap",
 "Accessibilité de l'installation en transport en commun des différents mode",
 'Observation Installation',
 "Date de changement d'état de la fiche d'enquête",
 "Date de création de la fiche d'enquête",
 'Installation hors-service',
 'EPCI INSEE',
 'Département Code',
 'Département Nom',
 'Région Code',
 'Région Nom',
 'EPCI Nom',
 'Bassin de vie Code',
 'Bassin de vie Nom',
 'Arrondissement Code',
 'Arrondissement Nom',
 'Densite Niveau',
 'Densite Catégorie',
 'Département Code Complet',
 'Réctorat Nom',
 'ZRR Simplifié',
 "Nom de l'équipement sportif",
 "Type d'équipement sportif",
 'Coordonnées',
 'Nom du propriétaire',


In [17]:
data_filtered = (data["Région Nom"] == "Hauts-de-France") | (data["Région Nom"] == "Occitanie") 

In [11]:
import plotly.express as px

REGIONS = ["Hauts-de-France", "Occitanie"]
SPORT_COLORS = {
    "Terrain de football": "blue",
    "Terrain de rugby": "red",
    "Terrain mixte": "green",
}

df = data[data["Région Nom"].isin(REGIONS)].copy()
df = df.dropna(subset=["Latitude", "Longitude"])

def build_region_map(region_name: str):
    tmp = df[df["Région Nom"] == region_name].copy()
    if tmp.empty:
        raise ValueError(f"No data found for region: {region_name}")
    
    center_lat = tmp["Latitude"].mean()
    center_lon = tmp["Longitude"].mean()

    fig = px.scatter_mapbox(
        tmp,
        lat="Latitude",
        lon="Longitude",
        hover_name="Nom de l'installation sportive",
        hover_data={
            "Commune nom": True,
            "Type d'équipement sportif": True,
            "Latitude": False,
            "Longitude": False,
        },
        color="Type d'équipement sportif",
        color_discrete_map=SPORT_COLORS,
        zoom=6,
        height=600,
        title=f"Football, Rugby and mixed Facilities – {region_name}",
    )

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_center={"lat": center_lat, "lon": center_lon},
        margin={"r": 0, "l": 0, "t": 50, "b": 0},
        legend_title_text="Sport type",
    )

    return fig

fig_hauts = build_region_map("Hauts-de-France")
fig_occ = build_region_map("Occitanie")

fig_hauts.show()
fig_occ.show()


C:\Users\Andraf\AppData\Local\Temp\ipykernel_14504\2641063933.py:21: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(
C:\Users\Andraf\AppData\Local\Temp\ipykernel_14504\2641063933.py:21: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [12]:
data[(data["Région Nom"] == "Occitanie") & 
     (data["Type d'équipement sportif"] == "Terrain de rugby")].shape[0]


552

In [13]:
import plotly.express as px
import pandas as pd

REGIONS = ["Hauts-de-France", "Occitanie"]
SPORT_COLORS = {
    "Terrain de football": "light blue",
    "Terrain de rugby": "red",
    "Terrain mixte": "green",
}

df = data[data["Région Nom"].isin(REGIONS)].copy()
df = df.dropna(subset=["Latitude", "Longitude"])

# Count facilities per region and sport type
counts = (
    df.groupby(["Région Nom", "Type d'équipement sportif"])
      .size()
      .reset_index(name="count")
)

# Stacked bar
fig = px.bar(
    counts,
    x="Région Nom",
    y="count",
    color="Type d'équipement sportif",
    color_discrete_map=SPORT_COLORS,
    title="Nombre de terrain par type par région",
    height=600
)

fig.update_layout(
    barmode="stack",
    xaxis_title="Région",
    yaxis_title="Nombre de terrains",
    legend_title="Type de sport",
    margin={"l": 40, "r": 40, "t": 50, "b": 40}
)

fig.show()


In [28]:
data.columns.tolist()

["Numéro de l'équipement sportif",
 "Numéro de l'installation sportive",
 "Date de l'enquête",
 "Nom de l'installation sportive",
 'SIRET Installation',
 'Adresse',
 'Code Postal',
 'Commune nom',
 'Commune INSEE',
 "Type de particularité de l'installation",
 'Unité Administrative Immatriculée (UAI)',
 "Accessibilité de l'installation en fonction du type handicap",
 "Accessibilité de l'installation en transport en commun des différents mode",
 'Observation Installation',
 "Date de changement d'état de la fiche d'enquête",
 "Date de création de la fiche d'enquête",
 'Installation hors-service',
 'EPCI INSEE',
 'Département Code',
 'Département Nom',
 'Région Code',
 'Région Nom',
 'EPCI Nom',
 'Bassin de vie Code',
 'Bassin de vie Nom',
 'Arrondissement Code',
 'Arrondissement Nom',
 'Densite Niveau',
 'Densite Catégorie',
 'Département Code Complet',
 'Réctorat Nom',
 'ZRR Simplifié',
 "Nom de l'équipement sportif",
 "Type d'équipement sportif",
 'Coordonnées',
 'Nom du propriétaire',


In [29]:
import numpy as np
import pandas as pd

# --- 1. Filtrer les terrains de rugby en Occitanie ---

equip_occ = data[data["Région Nom"] == "Occitanie"].copy()
equip_rugby = equip_occ[
    equip_occ["Activités"].astype(str).str.contains("Rugby", case=False, na=False)
].copy()

equip_rugby = equip_rugby.dropna(subset=["Latitude", "Longitude"])

# On passe en radians pour le Haversine
equip_rugby["lat_rad"] = np.radians(equip_rugby["Latitude"])
equip_rugby["lon_rad"] = np.radians(equip_rugby["Longitude"])

# --- 2. Communes d'Occitanie < 5 000 hab, avec coordonnées ---

comm = communes_pop.copy()
comm = comm[(comm["Région"] == "Occitanie") & (comm["p21_pop"] < 5000)].copy()
comm = comm.dropna(subset=["Latitude", "Longitude"])

comm["lat_rad"] = np.radians(comm["Latitude"])
comm["lon_rad"] = np.radians(comm["Longitude"])

# --- 3. Fonction de distance Haversine (en km) ---

def haversine_rad(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# --- 4. Matrice des distances communes -> stades de rugby ---

lat_comm = comm["lat_rad"].values[:, None]       # shape (N_communes, 1)
lon_comm = comm["lon_rad"].values[:, None]
lat_rug = equip_rugby["lat_rad"].values[None, :] # shape (1, N_stades)
lon_rug = equip_rugby["lon_rad"].values[None, :]

dist_matrix = haversine_rad(lat_comm, lon_comm, lat_rug, lon_rug)  # (N_communes, N_stades)

# --- 5. Distance minimale pour chaque commune ---

comm["dist_min_km"] = dist_matrix.min(axis=1)

# --- 6. Moyennes (simple et pondérée population) ---

mean_dist_simple = comm["dist_min_km"].mean()

mean_dist_weighted = np.average(comm["dist_min_km"], weights=comm["p21_pop"])

print(f"Distance minimale moyenne (non pondérée) : {mean_dist_simple:.2f} km")
print(f"Distance minimale moyenne pondérée par la population : {mean_dist_weighted:.2f} km")


NameError: name 'communes_pop' is not defined